In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
import joblib
from xgboost import XGBClassifier

In [43]:
df = pd.read_csv("D:\Data set\Bank Customer Churn Prediction.csv")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\admin\AppData\Local\Temp\ipykernel_16572\1025898505.py:1: SyntaxWarning: invalid escape sequence '\D'
  df = pd.read_csv("D:\Data set\Bank Customer Churn Prediction.csv")


In [44]:
df.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [45]:
print(df.shape)

(10000, 12)


In [46]:
print(df.columns)

Index(['customer_id', 'credit_score', 'country', 'gender', 'age', 'tenure',
       'balance', 'products_number', 'credit_card', 'active_member',
       'estimated_salary', 'churn'],
      dtype='object')


Preprocessing

In [47]:
df.drop(columns=['customer_id'], inplace=True)

In [48]:
# using dummies for columns with catagorical values

df = pd.get_dummies(df, columns=['country', 'gender'], drop_first=True)

In [49]:
df.head()

,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,country_Germany,country_Spain,gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,0,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,True,False


In [50]:
# split features from target variable

X = df.drop('churn', axis=1)
y = df['churn']

In [51]:
# test train split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [52]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8140


C:\Users\admin\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Add ML flow logging

In [53]:
import mlflow
import mlflow.sklearn

In [54]:
# # Enable MLflow auto-logging
# mlflow.set_experiment("Bank Churn Prediction")

# with mlflow.start_run():

#     # Model setup
#     model = LogisticRegression(max_iter=1000)
#     model.fit(X_train, y_train)

#     # Predictions
#     y_pred = model.predict(X_test)

#     # Metrics
#     acc = accuracy_score(y_test, y_pred)
#     prec = precision_score(y_test, y_pred)
#     rec = recall_score(y_test, y_pred)

#     # Log parameters & metrics
#     mlflow.log_param("model_type", "LogisticRegression")
#     mlflow.log_param("max_iter", 1000)
#     mlflow.log_metric("accuracy", acc)
#     mlflow.log_metric("precision", prec)
#     mlflow.log_metric("recall", rec)

#     # Log model
#     mlflow.sklearn.log_model(model, "model")

#     print("Run logged to MLflow.")

def train_and_log(model, name):
    with mlflow.start_run(run_name=name):
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        
        acc = accuracy_score(y_test, preds)
        prec = precision_score(y_test, preds)
        rec = recall_score(y_test, preds)
        
        mlflow.log_param("model_type", name)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)
        mlflow.log_metric("recall", rec)
        mlflow.sklearn.log_model(model, name)
        
        print(f"✅ {name} logged to MLflow")

# Start experiment
mlflow.set_experiment("Churn_Model_Comparison")

# Train models
train_and_log(LogisticRegression(max_iter=1000), "Logistic Regression")
train_and_log(RandomForestClassifier(n_estimators=100), "Random Forest")
train_and_log(XGBClassifier(use_label_encoder=False, eval_metric='logloss'), "XGBoost")

2025/06/15 19:01:00 INFO mlflow.tracking.fluent: Experiment with name 'Churn_Model_Comparison' does not exist. Creating a new experiment.
C:\Users\admin\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/06/15 19:01:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/15 19:01:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when log

✅ Logistic Regression logged to MLflow


2025/06/15 19:01:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/15 19:01:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Random Forest logged to MLflow


C:\Users\admin\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [19:01:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/06/15 19:01:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/15 19:02:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ XGBoost logged to MLflow


Save model.pkl

In [55]:
import joblib

# joblib.dump(model, "model.pkl")

best_model = RandomForestClassifier(n_estimators=100)
best_model.fit(X_train, y_train)
joblib.dump(best_model, "model.pkl")

['model.pkl']